# Titanic 필사 특별한 내용 정리

## 전처리 방법

In [ ]:
# 이상치 제거 방법 
def detect_outliers(df,n,features):
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        Q1 = np.percentile(df[col], 25) # 1분위수
        Q3 = np.percentile(df[col],75) # 3분위수
        IQR = Q3 - Q1
   
        outlier_step = 1.5 * IQR
        
        #  이상치 컬럼 index만 추출 
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # 리스트에 추가
        outlier_indices.extend(outlier_list_col)
        
    # 이상치 2개 이상인것들만 세서 row 넘버 리스트화
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# 4개 컬럼 함수 적용 
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])
# 이상치 제거
train = train.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

In [ ]:
# 결측값을 처리하는 방법

index_NaN_age = list(dataset["Age"][dataset["Age"].isnull()].index)
# null값이 있는 index만 추출해 list 화

for i in index_NaN_age :
    age_med = dataset["Age"].median()
    age_pred = dataset["Age"][((dataset['SibSp'] == dataset.iloc[i]["SibSp"]) & (dataset['Parch'] == dataset.iloc[i]["Parch"]) & (dataset['Pclass'] == dataset.iloc[i]["Pclass"]))].median()
    if not np.isnan(age_pred) : # nan이 아닐때 ? 
        dataset['Age'].iloc[i] = age_pred
    else :
        dataset['Age'].iloc[i] = age_med
        

In [ ]:
# 왜도 확인후 처리


# 왜도 확인
g = sns.distplot(dataset["Fare"], color="m", label="Skewness : %.2f"%(dataset["Fare"].skew()))
# 심하게 치우침
g = g.legend(loc="best")

# 0보다 큰건 다 log 처리
dataset["Fare"] = dataset["Fare"].map(lambda i: np.log(i) if i > 0 else 0)


In [ ]:
## 대체하기 
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
dataset["Title"] = dataset["Title"].astype(int)

In [ ]:
# 0,1 형식으로 변경 시켜줌 
dataset = pd.get_dummies(dataset, columns = ["Title"])
dataset = pd.get_dummies(dataset, columns = ["Embarked"], prefix="Em")

In [ ]:
# 결측값이면 X를 넣는 코드
Ticket = []
for i in list(dataset.Ticket):
    if not i.isdigit() :
        Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0]) #Take prefix
    else:
        Ticket.append("X")
        
dataset["Ticket"] = Ticket
dataset["Ticket"].head()

## 시각화

In [ ]:
# 수치형일떄 겹처 그리기

g = sns.kdeplot(train["Age"][(train["Survived"] == 0) & (train["Age"].notnull())], color="Red", shade = True)
g = sns.kdeplot(train["Age"][(train["Survived"] == 1) & (train["Age"].notnull())], ax =g, color="Blue", shade= True)
# kdeplot을 통해 null값이 아닌 분포를 생존별로 확인 
g.set_xlabel("Age")
g.set_ylabel("Frequency")
g = g.legend(["Not Survived","Survived"])
# 0~5는 생존 확실하게 한것을 볼 수 있음